In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count, date_format, desc, max, month, row_number
from pyspark.sql.window import Window


# inicializando a SparkSession
spark = SparkSession.builder.appName("EDA").getOrCreate()

# lendo dados de 'data/processed/'
df = spark.read.parquet("../data/processed/")

your 131072x1 screen size is bogus. expect trouble
24/10/10 15:07:32 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 172.24.246.235 instead (on interface eth0)
24/10/10 15:07:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/10 15:07:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/10 15:07:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# primeiras linhas do DataFrame
df.show()

+------+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+----------+---------------+-------------+
|    id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|      date|wind_spd_origin|wind_spd_dest|
+------+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+----------+---------------+-------------+
|293888|2013|    8| 15|  1459.0|          1459|      0.0|  1753.0|          1827|    -34.0|     DL|   161| N16065|   JFK| SEA|   318.0|    2422|  14|    59|2013-08-15 14:00:00|Delta Air Lines Inc.|2013-08-15|           NULL|         NULL|
|293889|2013|    8| 15|  1500.0|          14

In [3]:
df.filter(col("wind_spd_origin").isNotNull()).count()

3113

In [4]:
df.filter(col("wind_spd_dest").isNotNull()).count()

0

In [5]:
# schema da tabela
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)
 |-- dep_time: double (nullable = true)
 |-- sched_dep_time: long (nullable = true)
 |-- dep_delay: double (nullable = true)
 |-- arr_time: double (nullable = true)
 |-- sched_arr_time: long (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: long (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: double (nullable = true)
 |-- distance: long (nullable = true)
 |-- hour: long (nullable = true)
 |-- minute: long (nullable = true)
 |-- time_hour: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- wind_spd_origin: double (nullable = true)
 |-- wind_spd_dest: double (nullable = true)



In [6]:
# 1. número total de voos
total_flights = df.count()
print(f"Número total de voos: {total_flights}")

Número total de voos: 336776


In [7]:
# 2. número de voos cancelados (dep_time e arr_time nulos)
cancelled_flights = df.filter(col("dep_time").isNull() & col("arr_time").isNull()).count()
print(f"Número de voos cancelados: {cancelled_flights}")

Número de voos cancelados: 0


In [8]:
# 3. atraso médio na partida (dep_delay)
df_dep_delay = df.select("dep_delay")
avg_dep_delay = df_dep_delay.filter(col("dep_delay").isNotNull() & (col("dep_delay") > 0)).na.drop().select(avg("dep_delay")).collect()[0][0]
print(f"Atraso médio na partida: {avg_dep_delay} minutos")

Atraso médio na partida: 39.37323252771895 minutos


In [9]:
# 4. 5 aeroportos com maior número de pousos
df_airports_landings = df.select("id", "dest")
top5_airports_landings = df_airports_landings.groupBy("dest").agg(count("id").alias("num_landings")).orderBy(desc("num_landings")).limit(5)
top5_airports_landings.show()

+----+------------+
|dest|num_landings|
+----+------------+
| ORD|       17283|
| ATL|       17215|
| LAX|       16174|
| BOS|       15508|
| MCO|       14082|
+----+------------+



In [10]:
# 5. rota mais frequente (par origin-dest)
most_frequent_route = df.groupBy("origin", "dest").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(1)
most_frequent_route.show()

+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   JFK| LAX|      11262|
+------+----+-----------+



In [11]:
# 6. 5 companhias aéreas com maior tempo médio de atraso na chegada
df_carriers_avg_arr_delay = df.select("carrier", "arr_delay")
top5_carriers_avg_arr_delay = df_carriers_avg_arr_delay.na.drop().groupBy("carrier").agg(avg("arr_delay").alias("avg_arr_delay")).orderBy(desc("avg_arr_delay")).limit(5)
top5_carriers_avg_arr_delay.show()

+-------+------------------+
|carrier|     avg_arr_delay|
+-------+------------------+
|     F9|21.920704845814978|
|     FL|20.115905511811025|
|     EV| 15.79643108710965|
|     YV|15.556985294117647|
|     OO|11.931034482758621|
+-------+------------------+



In [12]:
# 7. dia da semana com maior número de voos
day_of_week_with_most_flights = df.withColumn("day_of_week", date_format("time_hour", "E")).groupBy("day_of_week").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(1)
day_of_week_with_most_flights.show()

+-----------+-----------+
|day_of_week|num_flights|
+-----------+-----------+
|        Mon|      50690|
+-----------+-----------+



In [13]:
# 8. percentual mensal dos voos que tiveram atraso na partida superior a 30 minutos
monthly_delays_over_30 = df.filter(col("dep_delay") > 30).groupBy(month("time_hour").alias("month")).agg((count("id") / df.count() * 100).alias("percent_delays_over_30"))
monthly_delays_over_30.orderBy(col("month")).show()

+-----+----------------------+
|month|percent_delays_over_30|
+-----+----------------------+
|    1|      1.14942870038245|
|    2|     1.319274532627028|
|    3|     1.535145022210609|
|    4|    1.5437560871320997|
|    5|    1.4784307670380312|
|    6|     1.997173195239566|
|    7|    2.1120863719504954|
|    8|    1.4027127823835428|
|    9|    0.8524954272275934|
|   10|    0.8709052901631944|
|   11|    0.7782621089388793|
|   12|    1.7507185785210346|
+-----+----------------------+



In [14]:
# 9. origem mais comum para voos que pousaram em Seattle (SEA)
most_common_origin_to_sea = df.filter(col("dest") == "SEA").groupBy("origin").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(1)
most_common_origin_to_sea.show()

+------+-----------+
|origin|num_flights|
+------+-----------+
|   JFK|       2092|
+------+-----------+



In [15]:
# 10. média de atraso na partida dos voos para cada dia da semana
df_avg_dep_delay_per_day = df.select("time_hour", "dep_delay")

avg_dep_delay_per_day = df_avg_dep_delay_per_day.na.drop() \
    .withColumn("day_of_week", date_format("time_hour", "E")) \
    .groupBy("day_of_week") \
    .agg(avg("dep_delay").alias("avg_dep_delay"))

avg_dep_delay_per_day.show()


+-----------+------------------+
|day_of_week|     avg_dep_delay|
+-----------+------------------+
|        Sun|11.589531801152422|
|        Mon|14.778936729330908|
|        Thu|16.148919990957108|
|        Sat| 7.650502333676133|
|        Wed|11.803512219083876|
|        Fri| 14.69605749486653|
|        Tue|10.631682565455652|
+-----------+------------------+



In [16]:
# 11. rota com maior tempo de voo médio
df_route_with_max_avg_airtime = df.select("origin", "dest", "air_time")
route_with_max_avg_airtime = df_route_with_max_avg_airtime.na.drop().groupBy("origin", "dest").agg(avg("air_time").alias("avg_air_time")).orderBy(desc("avg_air_time")).limit(1)
route_with_max_avg_airtime.show()

+------+----+-----------------+
|origin|dest|     avg_air_time|
+------+----+-----------------+
|   JFK| HNL|623.0877192982456|
+------+----+-----------------+



In [17]:
# 12. aeroporto de destino mais comum para cada origem
window_spec = Window.partitionBy("origin").orderBy(desc("num_flights"))
dest_most_common_per_origin = df.groupBy("origin", "dest").agg(count("id").alias("num_flights"))\
    .withColumn("rank", row_number().over(window_spec))\
    .filter(col("rank") == 1).drop("rank")

dest_most_common_per_origin.show()

+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   EWR| ORD|       6100|
|   JFK| LAX|      11262|
|   LGA| ATL|      10263|
+------+----+-----------+



In [18]:
# 13. 3 rotas com maior variação no tempo médio de voo
df_routes_with_max_variation_airtime = df.select("origin", "dest", "air_time")
routes_with_max_variation_airtime = df_routes_with_max_variation_airtime.na.drop().groupBy("origin", "dest").agg((max("air_time") - avg("air_time")).alias("air_time_variation")).orderBy(desc("air_time_variation")).limit(3)
routes_with_max_variation_airtime.show()

+------+----+------------------+
|origin|dest|air_time_variation|
+------+----+------------------+
|   JFK| SFO|142.59637439881612|
|   JFK| EGE|125.55445544554453|
|   JFK| LAX| 110.8489111927592|
+------+----+------------------+



In [19]:
# 14. média de atraso na chegada para voos com atraso na partida superior a 1 hora
df_avg_arr_delay_for_dep_delays_over_1hr = df.select("dep_delay", "arr_delay")
avg_arr_delay_for_dep_delays_over_1hr = df_avg_arr_delay_for_dep_delays_over_1hr.filter(col("dep_delay") > 60).na.drop().select(avg("arr_delay")).collect()[0][0]
print(f"Média de atraso na chegada para voos com atraso na partida superior a 1 hora: {avg_arr_delay_for_dep_delays_over_1hr}")

Média de atraso na chegada para voos com atraso na partida superior a 1 hora: 119.04880549963919


In [20]:
# 15. média de voos diários para cada mês
df_daily_avg_flights_per_month = df.select("id", "time_hour")
daily_avg_flights_per_month = df.withColumn("month", month("time_hour")).groupBy("month").agg((count("id") / 30).alias("avg_daily_flights"))
daily_avg_flights_per_month.orderBy(col("month")).show()

+-----+-----------------+
|month|avg_daily_flights|
+-----+-----------------+
|    1|900.1333333333333|
|    2|            831.7|
|    3|961.1333333333333|
|    4|944.3333333333334|
|    5|959.8666666666667|
|    6|941.4333333333333|
|    7|980.8333333333334|
|    8|977.5666666666667|
|    9|919.1333333333333|
|   10|962.9666666666667|
|   11|908.9333333333333|
|   12|937.8333333333334|
+-----+-----------------+



In [21]:
# 16. 3 rotas mais comuns com atrasos na chegada superiores a 30 minutos
top3_routes_with_arr_delays_over_30 = df.filter(col("arr_delay") > 30).groupBy("origin", "dest").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(3)
top3_routes_with_arr_delays_over_30.show()

+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   LGA| ATL|       1785|
|   LGA| ORD|       1538|
|   JFK| LAX|       1389|
+------+----+-----------+



In [22]:
# 17. principal destino para cada origem
window_spec = Window.partitionBy("origin").orderBy(desc("num_flights"))
main_dest_per_origin = df.groupBy("origin", "dest").agg(count("id").alias("num_flights"))\
    .withColumn("rank", row_number().over(window_spec))\
    .filter(col("rank") == 1).drop("rank")

main_dest_per_origin.show()

+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   EWR| ORD|       6100|
|   JFK| LAX|      11262|
|   LGA| ATL|      10263|
+------+----+-----------+



In [25]:
# 18. informações enriquecidas para os 5 voos com maior atraso na chegada
top5_delay_df = df.select("origin", "wind_spd_origin", "dest", "wind_spd_dest", "arr_delay")
top5_delay_df = top5_delay_df.na.drop(subset=["arr_delay"])
top5_delay_df.orderBy(col("arr_delay").desc()).limit(5).show()

+------+---------------+----+-------------+---------+
|origin|wind_spd_origin|dest|wind_spd_dest|arr_delay|
+------+---------------+----+-------------+---------+
|   JFK|           NULL| HNL|         NULL|   1272.0|
|   JFK|           NULL| CMH|         NULL|   1127.0|
|   EWR|           NULL| ORD|         NULL|   1109.0|
|   JFK|           NULL| SFO|         NULL|   1007.0|
|   JFK|           NULL| CVG|         NULL|    989.0|
+------+---------------+----+-------------+---------+

